In [1]:
from world_athletics_database import logger
import glob
import polars as pl

In [2]:
csv_files = glob.glob("../data/men/*.csv")
logger.info(csv_files)

[07/02/23 09:23:44] INFO     ['../data/men/one-mile.csv', '../data/men/4x400-metres-relay.csv',     ]8;id=424799;file:///var/folders/dy/v73twks152z4p6y_7yl9bmbc0000gn/T/ipykernel_9596/1632730425.py\1632730425.py]8;;\:]8;id=147346;file:///var/folders/dy/v73twks152z4p6y_7yl9bmbc0000gn/T/ipykernel_9596/1632730425.py#2\2]8;;\
                             '../data/men/4x1500-metres-relay.csv', '../data/men/shot-put.csv',                    
                             '../data/men/2000-metres-steeplechase.csv',                                           
                             '../data/men/discus-throw.csv',                                                       
                             '../data/men/5000-metres-race-walk.csv',                                              
                             '../data/men/5-kilometres-race-walk.csv',                                             
                             '../data/men/10-kilometres.csv',                                                      
                             '../data/men/10000-metres-race-walk.csv',                                             
                             '../data/men/20-kilometres-race-walk.csv',                                            
                             '../data/men/15-kilometres.csv', '../data/men/1500-metres.csv',                       
                             '../data/men/4x100-metres-relay.csv',                                                 
                             '../data/men/400-metres-hurdles.csv',                                                 
                             '../data/men/110-metres-hurdles.csv', '../data/men/100-metres.csv',                   
                             '../data/men/600-metres.csv',                                                         
                             '../data/men/3000-metres-steeplechase.csv',                                           
                             '../data/men/decathlon.csv', '../data/men/3000-metres.csv',                           
                             '../data/men/4x200-metres-relay.csv', '../data/men/two-miles.csv',                    
                             '../data/men/10-miles-road.csv',                                                      
                             '../data/men/50-kilometres-race-walk.csv',                                            
                             '../data/men/pole-vault.csv', '../data/men/hammer-throw.csv',                         
                             '../data/men/10000-metres.csv', '../data/men/5-kilometres.csv',                       
                             '../data/men/20-kilometres.csv', '../data/men/200-metres.csv',                        
                             '../data/men/triple-jump.csv', '../data/men/2000-metres.csv',                         
                             '../data/men/20000-metres-race-walk.csv',                                             
                             '../data/men/5000-metres.csv', '../data/men/1000-metres.csv',                         
                             '../data/men/400-metres.csv',                                                         
                             '../data/men/30-kilometres-race-walk.csv',                                            
                             '../data/men/300-metres.csv', '../data/men/marathon.csv',                             
                             '../data/men/3000-metres-race-walk.csv',                                              
                             '../data/men/javelin-throw.csv', '../data/men/half-marathon.csv',                     
                             '../data/men/high-jump.csv',                                                          
                             '../data/men/35-kilometres-race-walk.csv',                                            
                             '../data/men/4x800-metres-relay.csv', '../d

In [50]:
def transform_times(s: str | float):
    if isinstance(s, float):
        return s
    y = s.split(":")
    if not y[-1].isalnum():
        y[-1] = y[-1][:-1]
    if len(y) == 3:  # hh:mm:ss
        return 3600 * float(y[0]) + 60 * float(y[1]) + float(y[2])
    elif len(y) == 2:  # mm:ss
        return 60 * float(y[0]) + float(y[1])
    elif len(y) == 1:  # s.ms
        return float(y[0])
    else:
        raise Exception(f"Could not convert {s}")


def transform_score(s):
    return float(s)


def transform_rank(s):
    return float(s)


df_list = []

for csv in csv_files:
    if "relay" in csv:
        continue
    df = pl.read_csv(csv, infer_schema_length=10000)
    if "decathlon" in csv:
        df = df.take_every(2)  # every second row is the individual marks
        cols = df.columns
        df = df.select(c for c in cols[:11])  # throw away all individual marks

    df = df.with_columns(
        pl.col("Mark").apply(transform_times).cast(float),
        pl.col("Rank").apply(transform_rank).cast(int),
        pl.col("Results Score").apply(transform_score).cast(int),
        pl.col("Pos").cast(str),
        pl.lit(csv.split("/")[-1].replace(".csv", "").replace("-", " ").title()).alias(
            "Event"
        ),
    )
    df_list.append(df)

In [51]:
df = (
    pl.concat(
        [df for df in df_list],
        how="diagonal",
    )
    .drop("Unnamed: 6", "Unnamed: 7")
    .rename(mapping={"WIND": "Wind"})
    .with_columns(
        pl.col("DOB").str.strptime(pl.Date, format="%d %b %Y", strict=False),
        pl.col("Date").str.strptime(pl.Date, format="%d %b %Y", strict=False),
    )
)